In [ ]:
import codecs
import requests
from bs4 import BeautifulSoup
import random
from collections import OrderedDict

headers_of_request = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Host': 'www.wango.org'
}
regionIDs = OrderedDict([
    (5, 'South-America'),
    (9, 'Oceania'),
    (11, 'Western Africa'),
    (13, 'Central-America'),
    (14, 'Eastern Africa'),
    (15, 'Northern Africa'),
    (17, 'Middle Africa'),
    (18, 'Southern Africa'),
    (21, 'Northern-America'),
    (29, 'Caribbean'),
    (30, 'Eastern Asia'),
    (35, 'South-Eastern Asia'),
    (39, 'Southern-Europe'),
    (62, 'South-Central Asia'),
    (145, 'Western Asia'),
    (151, 'Eastern Europe'),
    (154, 'Nothern Europe'),
    (155, 'Western Europe')
])
countryIDs = OrderedDict([
    (5, '32,68,76,152,170,218,238,254,328,600,604,740,858,862,0'),
    (9, '16,36,90,184,242,258,296,316,520,540,548,554,570,574,580,583,584,585,598,612,772,776,798,876,882,0'),
    (11, '132,204,270,288,324,384,430,466,478,562,566,624,654,686,694,768,854,0'),
    (13, '84,188,222,320,340,484,558,591,0'),
    (14, '108,174,175,231,232,262,404,450,454,480,508,638,646,690,706,716,800,834,894,0'),
    (15, '12,434,504,732,736,788,818,0'),
    (17, '24,120,140,148,178,180,226,266,678,0'),
    (18, '72,426,516,710,748,0'),
    (21, '60,124,304,666,840,0'),
    (29, '28,44,52,92,136,192,212,214,308,312,332,388,474,500,530,533,630,659,660,662,670,780,796,850,0'),
    (30, '156,344,392,408,410,446,496,895,0'),
    (35, '96,104,116,360,418,458,608,626,702,704,764,0'),
    (39, '8,20,70,191,292,300,336,380,470,620,674,705,724,807,896,897,898,0'),
    (62, '4,50,64,144,356,364,398,417,462,524,586,762,795,860,0'),
    (145, '31,48,51,196,268,275,368,376,400,414,422,512,634,682,760,784,792,887,0'),
    (151, '100,112,203,348,498,616,642,643,703,804,0'),
    (154, '208,233,234,246,352,372,428,440,578,744,752,826,830,833,0'),
    (155, '40,56,250,276,438,442,492,528,756,0')
])

ccIDs = OrderedDict([
    (5, 'cc3300'),
    (9, '2586BA'),
    (11, 'ffcc00'),
    (13, '51ae63'),
    (14, 'AB875'),
    (15, '8AC2E8'),
    (17, 'ff9900'),
    (18, 'BFB07D'),
    (21, 'F85038'),
    (29, 'A5D0ED'),
    (30, 'F85038'),
    (35, 'cc3300'),
    (39, '2586BA'),
    (62, 'ffcc00'),
    (145, '51ae63'),
    (151, 'ABC875'),
    (154, '8AC2E8'),
    (155, 'ff9900')
])

s = requests.Session()
s.headers.update(headers_of_request)
BaseUrl = 'http://www.wango.org'
SubUrl = '/resources.aspx?section=ngodir'

output_db = []
an_entry = {
    'region_id': '',
    'region': '',
    'country': '',
    'name': '',
    'city': ''
}

for key, region in regionIDs:
    # Initial get and payload
    init_payload = OrderedDict([('sub','region'),
                                ('regionID',regionIDs[key]),('col', ccIDs[key])])
    ngo_initial_response = s.get(url=(BaseUrl + SubUrl),params=init_payload)

    # Filtered post.
    request_item = OrderedDict([
        ("Countries", countryIDs[key]),
        ("InterestAreas", "Environment"),
        ("currpage", 1),
    ])
    referer = {'Referer': ngo_initial_response.url}
    payload2 = OrderedDict([('sub','list'),('regionID', regionIDs[key])
                            ('col', ccIDs[key])])
    ngo_filtered_response = s.post(url=(BaseUrl + SubUrl),params=payload2,
                                   data=request_item, headers=referer)
    ngo_soup = BeautifulSoup(ngo_filtered_response.content,'lxml')
    filtered_content = ngo_soup.find('div',{'class': 'content'})
    
    #determining the number of pages
    num_of_pages = -1
    try:
        filtered_pages_content = filtered_content.find('td',{'valign': 'top'}).find('div',{'align': 'left'})
        for link in filtered_pages_content.findAll('a')
            num_of_pages += 1
    except:
        numf_of_pages = 1
        
    for i in range(1,num_of_pages):
        if (i > 1):
            request_item["currpage"] = i
            old_url = ngo_filtered_response.url
            referef['Referer'] = old_url
            ngo_filtered_response = s.post(url=(BaseUrl+SubUrl),params=payload2,
                                          data=request_item, headers=referer)
            ngo_soup = BeautifulSoup(ngo_filtered_response.content,'lxml')
            filtered_content = ngo_soup.find('div','class': 'context')
        
    # http://www.crummy.com/software/BeautifulSoup/bs4/doc/#going-down